In [1]:
import pandas as pd
import os
import re

# === Đọc dữ liệu gộp ===
file_path = os.path.join(os.getcwd(), "BaoCao_TongHop_Financials.xlsx")
df = pd.read_excel(file_path)

In [8]:
# Đổi tên cột thời gian
df = df.rename(columns={0: "Period"})

# Giữ lại kỳ Q4
df_q4 = df[df["Period"].astype(str).str.startswith("Q4/")].copy()
df_q4["Year"] = df_q4["Period"].str.extract(r"Q4/(\d{4})").astype(int)

# Chỉ lấy các dòng có chỉ tiêu cần thiết
df_q4 = df_q4[df_q4["1. Cho vay và cho thuê tài chính khách hàng"].notna()]

# Lấy đúng cột cần thiết
df_q4 = df_q4[["Bank", "Year", "1. Cho vay và cho thuê tài chính khách hàng"]]
df_q4 = df_q4.rename(columns={"1. Cho vay và cho thuê tài chính khách hàng": "Value"})

# Nếu có trùng Bank-Year thì chỉ giữ dòng đầu tiên
df_q4 = df_q4.drop_duplicates(subset=["Bank", "Year"])

# Tính tăng trưởng tín dụng
df_q4 = df_q4.sort_values(["Bank", "Year"])
df_q4["Growth_rate"] = df_q4.groupby("Bank")["Value"].pct_change() * 100

# Kết quả cuối cùng
df_q4_result = df_q4.dropna(subset=["Growth_rate"]).reset_index(drop=True)

# In danh sách ngân hàng có đủ dữ liệu
banks_with_growth = df_q4_result["Bank"].unique()
print("📌 Các ngân hàng có đủ dữ liệu để tính tăng trưởng tín dụng:\n")
print(sorted(banks_with_growth.tolist()))

# Hiển thị kết quả
df_q4_result

📌 Các ngân hàng có đủ dữ liệu để tính tăng trưởng tín dụng:

['ACB', 'CTG', 'HDB', 'MBB', 'STB', 'TCB', 'VCB', 'VPB']


,Bank,Year,Value,Growth_rate
0,ACB,2008,34833.0,10.806082
1,ACB,2009,62358.0,79.019895
2,ACB,2010,87195.0,39.829693
3,ACB,2011,102809.0,17.906990
4,ACB,2012,102802.0,-0.006809
...,...,...,...,...
103,VPB,2020,290816.0,13.077019
104,VPB,2021,355281.0,22.166937
105,VPB,2022,438338.0,23.377833
106,VPB,2023,566271.0,29.185925
